Imports for drawing images

In [ ]:
import pandas as pd
import cv2
from ast import literal_eval
import os

Imports for CNN

In [ ]:
BASE_SIZE = 256
size = 80

In [ ]:
def draw_cv2(raw_strokes, size=256, lw=6):
    """
    Takes in one array of the vectors from drawing and returns an image array
    """
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for stroke in raw_strokes:
        for i in range(len(stroke[0]) - 1):
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]), (stroke[0][i + 1], stroke[1][i + 1]), 255, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

In [ ]:
def create_images(filename):
    """
    Takes the filename of one category and creates a folder with all images for that category
    """
    folder = filename.split('.')[0]
    os.mkdir(folder)
    df = pd.read_csv(filename)
    df['drawing'] = df['drawing'].apply(literal_eval)
    for i, raw_strokes in enumerate(df.drawing.values):
        cv2.imwrite(f'{folder}/{i}.jpg', draw_cv2(raw_strokes, size=size))

In [ ]:
def create_all_images():
    """
    Creates images for all categories
    """
    training_set = './../train_simplified/'
    train_files = [os.join(training_set, f) for f in os.listdir(training_set) if os.isfile(os.join(training_set, f))]
    for filename in train_files:
        create_images(filename)